# YASA

This notebook demonstrates how to use YASA to perform **multi-channels slow-waves detection** from a NumPy array (example 1) or a MNE Raw object (example 2).

Please make sure to install YASA first by typing the following line in your terminal or command prompt:

`pip install --upgrade yasa`

**Important**
- The data must be a numpy array of shape *(n_channels, n_samples)*.
- The sampling frequency `sf` must be the same for all channels.
- A list of the channel names (`ch_names`) must be provided as well.
- The unit of the data must be $\mu V$. Note that the default unit in [MNE](https://martinos.org/mne/dev/generated/mne.io.Raw.html) is $V$. Therefore, if you use MNE, you must multiply your data by 1e6 (1 $V$ = 1,000,000 $\mu V$).

## Example 1: Using NumPy

To illustrate the multi-channel slow-waves detection, we load a full-night 3-channels dataset (Cz, Fz, Pz) sampled at 100 Hz. The data is in compressed NumPy format (*.npz*).

In [1]:
import yasa
import numpy as np

# Load data
f = np.load('data_full_6hrs_100Hz_Cz+Fz+Pz.npz')
data, chan = f['data'], f['chan']
sf = 100.
times = np.arange(data.size) / sf

print(data.shape, chan)
print(np.round(data[:, 0:5], 3))

C:\Users\Raphael\AppData\Local\Continuum\anaconda3\lib\site-packages\numba\decorators.py:146: RuntimeWarning: Caching is not available when the 'parallel' target is in use. Caching is now being disabled to allow execution to continue.
  warnings.warn(msg, RuntimeWarning)


(3, 2161058) ['Cz' 'Fz' 'Pz']
[[15.797 22.307 39.922 25.657 27.094]
 [16.896 26.385 40.966 21.833 24.456]
 [ 5.899 14.297 36.592 26.094 23.395]]


*************

**Applying the detection**

To apply the multi-channel detection, we use the [sw_detect_multi](https://raphaelvallat.com/yasa/build/html/generated/yasa.spindles_detect_multi.html#yasa.spindles_detect_multi) function.

In [2]:
sw = yasa.sw_detect_multi(data, sf, ch_names=chan)
print(sw.shape[0], 'slow-waves detected.')
sw.head().round(3)

8353 slow-waves detected.


,Start,NegPeak,MidCrossing,PosPeak,End,Duration,ValNegPeak,ValPosPeak,PTP,Slope,Frequency,Channel,IdxChannel
0,79.94,80.11,80.79,80.91,81.27,1.33,-56.394,25.713,82.108,120.746,0.752,Cz,0
1,91.25,91.72,92.03,92.17,92.33,1.08,-48.297,29.271,77.568,250.219,0.926,Cz,0
2,99.11,99.91,100.26,100.45,100.59,1.48,-59.855,91.428,151.283,432.237,0.676,Cz,0
3,100.59,101.01,101.19,101.31,101.44,0.85,-127.073,66.482,193.555,1075.303,1.176,Cz,0
4,426.67,427.09,427.24,427.36,427.65,0.98,-109.979,47.775,157.753,1051.689,1.020,Cz,0


In [3]:
# We print the number of slow-waves detected per channel, as well as the mean slow-waves properties per channel.
display(sw['Channel'].value_counts())
display(sw.groupby('Channel').mean().round(2))

Fz    2863
Cz    2831
Pz    2659
Name: Channel, dtype: int64

,Start,NegPeak,MidCrossing,PosPeak,End,Duration,ValNegPeak,ValPosPeak,PTP,Slope,Frequency,IdxChannel
Channel,,,,,,,,,,,,
Cz,9129.35,9129.63,9129.90,9130.11,9130.41,1.06,-89.35,60.23,149.58,684.00,1.04,0
Fz,8854.94,8855.21,8855.46,8855.67,8855.99,1.05,-92.79,55.98,148.77,703.64,1.04,1
Pz,8955.23,8955.51,8955.79,8956.03,8956.33,1.10,-84.61,60.91,145.52,635.01,1.00,2


In [4]:
# For plotting purposes, we can easily extract a boolean vector that has the same size as the data
bool_vector = yasa.get_bool_vector(data, sf, sw)
print(bool_vector.sum(1))
print(bool_vector)

[302270 302275 293299]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


*************

## Example 2: Using a Raw object from MNE-Python

This example demonstrates how to manipulate [MNE Raw object](https://mne-tools.github.io/stable/generated/mne.io.Raw.html#mne.io.Raw). The MNE package has several [functions](https://mne-tools.github.io/stable/python_reference.html#module-mne.io) to load the most standard EEG file formats (EDF, BrainVision, EEGLab, FieldTrip...).

### Load using MNE
For the sake of this example, we'll load a PSG file encoded in the native MNE format (*.fif) using the [mne.io.read_raw_fif](https://martinos.org/mne/stable/generated/mne.io.read_raw_fif.html) function.

In [5]:
import mne

# Load the raw object
raw = mne.io.read_raw_fif('sub-02_mne_raw.fif', preload=True)

Opening raw data file sub-02_mne_raw.fif...
    Range : 0 ... 293999 =      0.000 ...  2939.990 secs
Ready.
Reading 0 ... 293999  =      0.000 ...  2939.990 secs...


In [6]:
# Let's have a look at the data
print('Chan =', raw.ch_names)
print('Sampling frequency =', raw.info['sfreq'])
print('Data shape =', raw._data.shape)

Chan = ['F3', 'F4', 'C3', 'C4', 'O1', 'O2', 'EOG1', 'EOG2', 'EMG1']
Sampling frequency = 100.0
Data shape = (9, 294000)


### Applying YASA

In [7]:
# Keep only the channels of interests
raw_eeg = raw.copy().pick_types(eeg=True).drop_channels(['O1', 'O2'])
print('Chan =', raw_eeg.ch_names)

Chan = ['F3', 'F4', 'C3', 'C4']


In [8]:
# Multi-channel slow-waves detection
# Note that since we're using a MNE Raw object, there is no need
# to manually specify the sampling frequency and channel names.
sw = yasa.sw_detect_multi(raw_eeg)
print(sw.shape[0], 'slow-waves detected.')
sw.head().round(3)

305 slow-waves detected.


,Start,NegPeak,MidCrossing,PosPeak,End,Duration,ValNegPeak,ValPosPeak,PTP,Slope,Frequency,Channel,IdxChannel
0,0.35,0.69,0.82,0.95,1.14,0.79,-76.507,40.610,117.117,900.902,1.266,F3,0
1,15.60,15.77,16.04,16.19,16.56,0.96,-97.910,149.235,247.145,915.353,1.042,F3,0
2,38.04,38.20,38.35,38.45,38.56,0.52,-49.548,27.547,77.095,513.964,1.923,F3,0
3,38.56,38.88,38.98,39.11,39.26,0.70,-140.918,184.772,325.690,3256.902,1.429,F3,0
4,196.55,196.85,196.99,197.12,197.66,1.11,-77.365,45.152,122.517,875.122,0.901,F3,0


In [9]:
bool_vector = yasa.get_bool_vector(raw_eeg, detection=sw)
print(bool_vector.sum(1))
print(bool_vector)

[ 9732 11304  6322  5396]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
